In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
from cooc import *
#rom glove_template import *
from pickle_vocab import *

In [24]:
with open('data_/pos_train.txt') as f:
    pos = f.readlines()

In [25]:
with open('data_/neg_train.txt') as f:
    neg = f.readlines()

In [53]:
train=pos+neg

In [54]:
train_target=[':)']*len(pos)+[':(']*len(neg)

In [55]:
import pandas as pd

full_train={'data':train,
           'target':train_target}
full_train=pd.DataFrame(full_train)

,data,target
0,<user> i dunno justin read my mention or not ....,:)
1,"because your logic is so dumb , i won't even c...",:)
2,""" <user> just put casper in a box ! "" looved t...",:)
3,<user> <user> thanks sir > > don't trip lil ma...,:)
4,visiting my brother tmr is the bestest birthda...,:)
...,...,...
199995,can't wait to fake tan tonight ! hate being pa...,:(
199996,<user> darling i lost my internet connection ....,:(
199997,kanguru defender basic 4 gb usb 2.0 flash driv...,:(
199998,rizan is sad now\n,:(


In [56]:
from sklearn.model_selection import train_test_split

train , test=train_test_split(full_train)

### Bayes Classifier

In [57]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.data)
X_train_counts.shape

(150000, 85007)

In [58]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(150000, 85007)

In [59]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train.target)

In [60]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(train.data, train.target)

In [61]:
import numpy as np
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)

0.76782

### SVM

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=1000, random_state=42))])

text_clf_svm = text_clf_svm.fit(train.data, train.target)
predicted_svm = text_clf_svm.predict(test.data)
np.mean(predicted_svm == test.target)